# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [7]:
# Write your code below.
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [8]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [9]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA") ## uses the .env file


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [18]:
# Write your code below.
from glob import glob ## this will read files (parquet files above)

parq_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True) ## double asterisk ** means it will add all subfolders
dd_pr = dd.read_parquet(parq_files).set_index("Ticker").reset_index() ## reset index to make Ticker a column again
#dd_pr.compute()

## add lag for variables, Close and Adj Close, but first sort by Ticker and Date
dd_pr = dd_pr.sort_values(by=["Ticker","Date"]) ## sort by Ticker and Date
dd_pr1 = dd_pr.groupby('Ticker', group_keys=False).apply(
                    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1)) ## apply a function lamda that will shift the close column by 1
)
dd_pr2 = dd_pr1.groupby('Ticker', group_keys=False).apply(
                    lambda x: x.assign(Adj_Close_lag_1 = x['Adj Close'].shift(1)) ## apply a function lamda that will shift the close column by 1
)
#dd_pr1.compute() ## this will show the results

## add return based on Close
dd_pr2['returns'] = (dd_pr2['Close']/dd_pr2['Close_lag_1']) - 1

## add hi-lo range variable
dd_pr2['hi_lo_range'] = dd_pr2['High'] - dd_pr2['Low']

## result should be called dd_feat dataframe
dd_feat = dd_pr2.compute() ## this will show the results


C:\Users\Thurai\AppData\Local\Temp\ipykernel_15752\4267351420.py:10: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_pr1 = dd_pr.groupby('Ticker', group_keys=False).apply(
C:\Users\Thurai\AppData\Local\Temp\ipykernel_15752\4267351420.py:13: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_pr2 = dd_pr1.groupby('Ticker', group_keys=False).apply(


In [19]:
dd_feat

Price,Ticker,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
0,ROL,2000-01-03 00:00:00+00:00,0.637498,0.888889,0.888889,0.855967,0.877915,416897.0,2000,NaN,NaN,NaN,0.032922
1,ROL,2000-01-04 00:00:00+00:00,0.632251,0.881573,0.888889,0.881573,0.881573,198197.0,2000,0.888889,0.637498,-0.008230,0.007316
2,ROL,2000-01-05 00:00:00+00:00,0.629628,0.877915,0.885231,0.870599,0.881573,534790.0,2000,0.881573,0.632251,-0.004149,0.014632
3,ROL,2000-01-06 00:00:00+00:00,0.634874,0.885231,0.885231,0.877915,0.877915,370765.0,2000,0.877915,0.629628,0.008333,0.007316
4,ROL,2000-01-07 00:00:00+00:00,0.640121,0.892547,0.892547,0.885231,0.892547,85430.0,2000,0.885231,0.634874,0.008265,0.007316
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,TPR,2025-01-23 00:00:00+00:00,73.620003,73.620003,73.820000,72.550003,72.930000,4276300.0,2025,73.339996,73.339996,0.003818,1.269997
14,TPR,2025-01-24 00:00:00+00:00,74.650002,74.650002,75.779999,73.529999,73.620003,3857700.0,2025,73.620003,73.620003,0.013991,2.250000
15,TPR,2025-01-27 00:00:00+00:00,72.510002,72.510002,73.949997,71.769997,73.830002,4528800.0,2025,74.650002,74.650002,-0.028667,2.180000
16,TPR,2025-01-28 00:00:00+00:00,73.790001,73.790001,74.000000,72.260002,72.500000,3481400.0,2025,72.510002,72.510002,0.017653,1.739998


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [20]:
# Write your code below.
# convert Dask dataframe to pandas dataframe
import pandas as pd
df = pd.DataFrame(dd_feat)

# add a new column containing the moving average of returns using a window of 10 days
df['returns_MA_10'] = df.groupby('Ticker')['returns'].transform(lambda x: x.rolling(10).mean())

In [21]:
df

Price,Ticker,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_MA_10
0,ROL,2000-01-03 00:00:00+00:00,0.637498,0.888889,0.888889,0.855967,0.877915,416897.0,2000,NaN,NaN,NaN,0.032922,NaN
1,ROL,2000-01-04 00:00:00+00:00,0.632251,0.881573,0.888889,0.881573,0.881573,198197.0,2000,0.888889,0.637498,-0.008230,0.007316,NaN
2,ROL,2000-01-05 00:00:00+00:00,0.629628,0.877915,0.885231,0.870599,0.881573,534790.0,2000,0.881573,0.632251,-0.004149,0.014632,NaN
3,ROL,2000-01-06 00:00:00+00:00,0.634874,0.885231,0.885231,0.877915,0.877915,370765.0,2000,0.877915,0.629628,0.008333,0.007316,NaN
4,ROL,2000-01-07 00:00:00+00:00,0.640121,0.892547,0.892547,0.885231,0.892547,85430.0,2000,0.885231,0.634874,0.008265,0.007316,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,TPR,2025-01-23 00:00:00+00:00,73.620003,73.620003,73.820000,72.550003,72.930000,4276300.0,2025,73.339996,73.339996,0.003818,1.269997,0.009080
14,TPR,2025-01-24 00:00:00+00:00,74.650002,74.650002,75.779999,73.529999,73.620003,3857700.0,2025,73.620003,73.620003,0.013991,2.250000,0.008742
15,TPR,2025-01-27 00:00:00+00:00,72.510002,72.510002,73.949997,71.769997,73.830002,4528800.0,2025,74.650002,74.650002,-0.028667,2.180000,0.007889
16,TPR,2025-01-28 00:00:00+00:00,73.790001,73.790001,74.000000,72.260002,72.500000,3481400.0,2025,72.510002,72.510002,0.017653,1.739998,0.010250


Please comment:

Was it necessary to convert to pandas to calculate the moving average return?
- No, it is not necessary to convert to pandas to calculate the moving average return.

Would it have been better to do it in Dask? Why?
- It would have been better to do it in dask because dask has built-in support for rolling window operations using .rolling() similar to Pandas.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [X] Created a branch with the correct naming convention.
- [X] Ensured that the repository is public.
- [X] Reviewed the PR description guidelines and adhered to them.
- [X] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.